In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import keras
import os, sys, math
import tensorflow as tf
import pickle

from tensorflow.keras import layers
from tensorflow.keras import activations

from keras.models import Sequential
from keras.models import load_model

from keras import backend as K 

from keras.layers.convolutional import Conv2D # to add convolutional layers

from keras.layers import Conv2DTranspose
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import GlobalAveragePooling2D
from keras.layers import Reshape
from keras.layers import Multiply

from scipy.io import loadmat

device_name = tf.test.gpu_device_name()
#if device_name != '/device:GPU:0':
#  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

class SE_RESNet(layers.Layer): 
  def __init__(self, strides=(1,1) , kernel_size=3, filters=128, ratio=16):   
    super(SE_RESNet, self).__init__() 

    self.conv1 = Conv2D(filters, kernel_size, strides, padding='same', activation='relu')
    self.bn1 = layers.BatchNormalization()

    self.conv2 = Conv2D(filters, kernel_size, strides, padding='same', activation='relu')
    self.bn2 = layers.BatchNormalization()

    self.aver = GlobalAveragePooling2D()

    self.dense1 = Dense(filters // ratio, activation='relu')
    self.dense2 = Dense(filters, activation='sigmoid')
    self.reshape = Reshape([1, 1, filters])

    self.mult = Multiply()


  def call(self, input_tensor, training=False):
    x = self.conv1(input_tensor)
    x = self.bn1(x, training=training)
    
    x = self.conv2(x)
    x = self.bn2(x, training=training)

    se = self.aver(x)
    se = self.dense1(se)
    se = self.dense2(se)
    se = self.reshape(se)
    
    x = self.mult([x, se])

    return x + input_tensor

def myModel(train = 0):
    
    # create model
    model = Sequential()

    layer1 = Conv2D(32, 9, strides=2, padding='same', input_shape=(24, 32, 5))
    layer2 = layers.BatchNormalization()
    if train>0:
      layer1.trainable = False
      layer2.trainable = False
    model.add(layer1)
    model.add(layers.Activation(activations.relu))
    model.add(layer2)

    layer3 = Conv2D(64, 3, strides=2, padding='same')
    layer4 = layers.BatchNormalization()
    if train>1:
      layer3.trainable = False
      layer4.trainable = False
    model.add(layer3)
    model.add(layers.Activation(activations.relu))
    model.add(layer4)

    layer5 = Conv2D(128, 3, padding='same')
    layer6 = layers.BatchNormalization()
    if train>2:
      layer5.trainable = False
      layer6.trainable = False
    model.add(layer5)
    model.add(layers.Activation(activations.relu))
    model.add(layer6)

    layer7 = SE_RESNet()
    model.add(layer7)

    layer8 = SE_RESNet()
    model.add(layer8)

    layer9 = SE_RESNet()
    model.add(layer9)

    layer10 = SE_RESNet()
    model.add(layer10)

    layer11 = SE_RESNet()
    model.add(layer11)

    layer12 = Conv2DTranspose(64, 3, strides=2, activation='relu', padding='same')
    layer13 = layers.BatchNormalization()
    if train>8:
      layer12.trainable = False
      layer13.trainable = False   
    model.add(layer12)
    model.add(layer13)  

    layer14 = Conv2DTranspose(32, 3, strides=2, activation='relu', padding='same')
    layer15 = layers.BatchNormalization()
    if train>9:
      layer14.trainable = False
      layer15.trainable = False   
    model.add(layer14)
    model.add(layer15)
    

    layer16 = Conv2D(1, 9, padding='same', activation='relu')
    if train>10:
      layer16.trainable = False
    model.add(layer16)
    
    # Compile model
    model.compile(optimizer='adam', loss='mean_squared_error')

    return model

# 0 -> sin normalizar, 1 -> min-max, 2 -> media y desviacion tipica
def initialize_data(data, type=0):
  #train = train[0:2000,:]
  train = np.copy(data)
  np.random.seed(0)
  np.random.shuffle(train)

  trainRate = 80 #80
  height = 24
  width = 32
  resolution = height*width

  felement = math.floor((train.shape[0]*trainRate)/100)

  # min-max
  min = list()
  max = list()

  img = list()
  test_img = list()

  # A la hora de normalizar se convierte a 1 toda desviación típica que sea igual a 0
  for i in range(0, 6):
    
    # Muestras de entrenamiento
    temp_img = train[:felement,resolution*i:resolution*(i+1)]

    # sin normalizar
    if (type == 0):
      img.append(temp_img)

      # Muestras de test
      temp_img = train[felement:,resolution*i:resolution*(i+1)]
      #test_img.append((temp_img - mean[i]) / std[i])
      test_img.append(temp_img)

    # min-max
    if (type == 1):
      if (i != 5):
        temp_min = np.min(temp_img, 0)
        temp_max = np.max(temp_img, 0)
        ind = temp_max==0
        temp_max[ind] = 1
        min.append(temp_min)
        max.append(temp_max)
        img.append((temp_img - min[i]) / (max[i] - min[i]))

        # Muestras de test
        temp_img = train[felement:,resolution*i:resolution*(i+1)]
        test_img.append((temp_img - min[i]) / (max[i] - min[i]))
      else:
        img.append(temp_img)

        # Muestras de test
        temp_img = train[felement:,resolution*i:resolution*(i+1)]
        #test_img.append((temp_img - mean[i]) / std[i])
        test_img.append(temp_img)

  input = np.moveaxis(np.array([img[0], img[1], img[2], img[3], img[4]]), 0, 2) 
  input = input.reshape(input.shape[0], 24, 32,input.shape[2])
  output = img[5]
  output = output.reshape(output.shape[0], 24, 32)

  test_input = np.moveaxis(np.array([test_img[0], test_img[1], test_img[2], test_img[3], test_img[4]]), 0, 2)
  test_input = test_input.reshape(test_input.shape[0], 24, 32,test_input.shape[2])
  test_output = test_img[5]
  test_output = test_output.reshape(test_output.shape[0], 24, 32)

  return input, output, test_input, test_output, min, max

def normalizar_imagen(image, min, max):  
  # normalizamos la entrada min-max
  img = []
  for i in range(0, 5):
    temp_img = image[:,:,i].flatten()
    img.append((temp_img - min[i]) / (max[i] - min[i]))

  input_img = np.moveaxis(np.array([img[0], img[1], img[2], img[3], img[4]]), 0, 1) 
  input_img = input_img.reshape(24, 32,input_img.shape[1])
  input_img = np.array(input_img)

  return input_img

def dist_betw_images(x, y):
  return abs(np.sum(x**2 - y**2))

if 'google.colab' in sys.modules: # Colab-only Tensorflow version selector
  %tensorflow_version 2.x
  
print("Tensorflow version " + tf.__version__)
AUTO = tf.data.experimental.AUTOTUNE
# Detect hardware
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
except ValueError:
  tpu = None
  gpus = tf.config.experimental.list_logical_devices("GPU")
    
# Select appropriate distribution strategy for hardware
if tpu:
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.experimental.TPUStrategy(tpu)
  print('Running on TPU ', tpu.master())  
elif len(gpus) > 0:
  #strategy = tf.distribute.MirroredStrategy(gpus) # this works for 1 to multiple GPUs
  print('Running on ', len(gpus), ' GPU(s) ')
  strategy=tf.distribute.OneDeviceStrategy(gpus[0])
else:
  strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  print('Running on CPU')
 
# How many accelerators do we have ?
print("Number of accelerators: ", strategy.num_replicas_in_sync)

Found GPU at: /device:GPU:0
Tensorflow version 2.5.0
Running on  1  GPU(s) 
Number of accelerators:  1


In [ ]:
data = np.load("/content/drive/MyDrive/tfg/testDataNumpy10k.npy")
tdata = data[:,:,:,:]

np.random.seed(2)
np.random.shuffle(tdata)

tdata = tdata[:,:,:,:]

trainRate = 80 #80

felement = math.floor((tdata.shape[0]*trainRate)/100)
felement = (math.floor(felement/256))*256

train = tdata[:felement,:,:,:]
test = tdata[felement:,:,:,:]

inp = train[:,:,:,:5]
outp = train[:,:,:,5]
inpt = test[:,:,:,:5]
outpt = test[:,:,:,5]